# 주제 :  인스타그램 크롤링 + 시각화(워드클라우드)
# 목적 : 시험
# 제출처 : 비트캠프

# 제목: 인스타그램 크롤링 + 시각화(워드클라우드)



In [2]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))


In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.size'] = 12.0
plt.rcParams['font.family'] = 'Malgun Gothic'

# 데이터 분석 과정 

- 모듈 임포트 
- 데이타 수집 (학습데이타)
- 데이터 탐색
- 데이타 전처리 
- 데이터 학습 (모델화)
- 데이터 예측 (테스트데이타)
- 데이터 테스트 (정답률)
- + 시각화 화면 제시 

In [4]:
# 학습기 모듈 
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [5]:
# 학습데이터와 데스트 전용 데이터로 분할하는 메서드 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터수집
#### 클로링 방법을 이용하여 수집 함
#### py 로 작성하여 됨.

In [6]:
# 필요 라이브러리 인스톨 및 임포트

!pip install selenium==3.11.0
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import csv 

# 경기도 행정구역 리스트
gg = [
  '가평군',  '고양시',  '과천시', '광명시',
  '광주시',  '구리시',  '군포시', '김포시',
  '남양주시', '동두천시', '부천시', '성남시',
  '수원시',  '시흥시',  '안산시', '안성시',
  '안양시',  '양주시',  '양평군', '여주시',
  '연천군',  '오산시',  '용인시', '의왕시',
  '의정부시', '이천시',  '파주시', '평택시',
  '포천시',  '하남시',  '화성시'
]

# 나중에 결과 저장할 딕셔너리 생성
insta_dict = {}

# selenium 이용해서 드라이버 지정하고 url 불러오기
# keyword 를 포함한 url로 쓰기
driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 

#keyword를 for문으로 돌릴 수 있으나 나중에 driver에서 오류가 날 가능성이 높음.
keyword = gg[1]
insta_url = "https://www.instagram.com"
search_url = insta_url + "/explore/tags/" + keyword
driver.get(search_url)
driver.maximize_window() #창 크게
time.sleep(3)


## 자동 로그인

# 아이디, 비번 변수에 저장
insta_id = "ggdata_semi_5"
insta_pw = "ggdata"

find_login = driver.find_elements_by_css_selector(
    "label.f0n8F input._2hvTZ.pexuQ.zyHYP")
# id 입력
input_id = find_login[0]
input_id.click()
input_id.send_keys(insta_id)

# pw 입력
input_password = find_login[1]
input_password.click()
input_password.send_keys(insta_pw)

# 로그인 버튼 클릭 click
login = driver.find_element_by_css_selector("button.sqdOP.L3NKy.y3zKF")
login.click()

## 로그인 시 팝업 창 제거
# 로그인 저장 팝업 제거
later_button = driver.find_elements_by_class_name("sqdOP")
later_button[0].click()

# 알림창 제거
alert = driver.find_elements_by_css_selector("button.aOOlW")
alertLater = alert[1]
alertLater.click()

# 브라우저가 잘 작동해서 검색이 가능하다면,
# 검색창에 검색어 입력하기 : 창 끄기가 chrome에서는 작동하지 않아서 검색어는 url에 바로 주기로
# keyword = "광명시"
# # search bar
# time.sleep(1)
# searchBar = driver.find_element_by_css_selector("div.eyXLr.wUAXj")
# searchBar.click()

# # search input
# searchKeyword = driver.find_element_by_css_selector("input.XTCLo.x3qfX")
# searchKeyword.click()
# searchKeyword.send_keys(keyword)

## 첫번째 사진 클릭 후 텍스트 가져오기~다음버튼 클릭하기 반복하기
# 첫번째 사진 클릭하기
driver.find_element_by_class_name("v1Nh3").click()

#크롤링한 결과 넣을 리스트, 그거 모을 딕트 생성
texts = []


#반복문. 10단위로 끝날때마다 알림프린트
i = 0
while i < 100:
    i += 1
    # # 게시물 텍스트 가져오기
    text = driver.find_element_by_class_name("C4VMK").text
    texts.append(text)
    time.sleep(1)
    # 다음 버튼 클릭하기
    driver.find_element_by_class_name("_65Bje").click()
    time.sleep(2)
    if i % 10 == 0:
        print("{0}번 끝".format(i))
# 결과 dict에 저장하기
insta_dict[ keyword ] = texts

# 창 나가기
actions = webdriver.ActionChains(driver).send_keys(Keys.ESCAPE)
actions.perform()


# 완성 딕셔너리 확인
insta_dict

# 딕셔너리를 csv 파일에 저장하기
with open('insta_texts.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = ['index','city','insta_text'])
    writer.writeheader()
    for key, value in insta_dict.items():
        i = 0
        writer.writerow({'index': i , 'city': key, 'insta_text': value})
        i += 1


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 21.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


WebDriverException: Message: '(driver) chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


# 데이타 전처리 

In [ ]:
# 전처리 함수 작성하자
import numpy as np
import pandas as pd
from unicodedata import normalize


#인덱스에 필요한 gg 리스트
gg = [
  '가평군',  '고양시',  '과천시', '광명시',  
  '광주시',  '구리시',  '군포시', '김포시',
  '남양주시', '동두천시', '부천시', '성남시',
  '수원시',  '시흥시',  '안산시', '안성시',
  '안양시',  '양주시',  '양평군', '여주시',
  '연천군',  '오산시',  '용인시', '의왕시',
  '의정부시', '이천시',  '파주시', '평택시',
  '포천시',  '하남시',  '화성시'
]
# 정제된 텍스트 담을 데이터 프레임 미리 만들어두기 : column 만 주기
df_after = pd.DataFrame(columns=['city', 'insta_text'])

# # 크롤링해서 저장한 데이터 불러와서 before에 저장. 
df_before = pd.read_excel('insta_merged.xlsx', encoding='utf-8')

# 데이터 인덱스 별 텍스트에 대해 쓸모없는 부분 처리해주고 정규화 해주고 after df 에 append해서 새로운 df 만들기 
for i in range(31):
    before = df_before['text'].loc[i]
    before = before.replace('\\n','')
    before = before.replace('[', '')
    before = before.replace(']', '')
    before = before.replace('#', '')
    before = before.replace('_', ' ')
    before = before.replace("'", '')
    after = normalize('NFC', before) #konlpy가 읽을 수 있게 인스타 텍스트 형식 정규화
    
    # 처리가 끝난 after를 시 이름과 함께 새로운 after 데이터 프레임에 append 해주기
    row = [gg[i], after]
    row_series = pd.Series(row, index = df_after.columns)
    df_after = df_after.append(row_series, ignore_index=True)

# 만든 새 데이터프레임을 엑셀로 저장
df_after.to_excel("insta_normalized.xlsx")

In [ ]:
# 필요 라이브러리 받기
import konlpy
from konlpy.tag import Okt
okt = Okt()
# 정규화 한 데이터 불러오기
import numpy as np
import pandas as pd

df_normal = pd.read_excel('insta_normalized.xlsx', encoding='utf-8')
df_normal.head(2)

# 테그된 단어 리스트 담을 데이터 프레임 미리 만들어두기
df_tagged = pd.DataFrame(columns=['city', 'tagged'])
df_tagged.head()

# 31개 시에 대해 반복하기
# okt class가 잘 동작하려면 800개씩 끊어서 태깅하고 나중에 다시 extend로 한 리스트에 합치기.
# 테깅이 끝나면 데이터프레임에 append 해주기

for i in range(31):
    #전체 택스트
    whole = df_normal['insta_text'].loc[i]
    long = len(whole)
    n = long // 800 # 몇번돌아야 하는지.. n+1번 만약 1601개면 3번(1-800,2-1600)
    all_tagged = [] # 전체리스트. 각 파트 돌때마다 담아줄거야
    for cut in range(n+1):
        part = whole[cut*800 : (cut+1)*800]
        raw_pos_tagged = okt.pos(part, norm=True, stem=True)
        #print(raw_pos_tagged)
        all_tagged.extend(raw_pos_tagged)

    # 데이터프레임에 row 기준으로 업데이트
    row = [gg[i], all_tagged]
    row_series = pd.Series(row, index = df_tagged.columns)
    df_tagged = df_tagged.append(row_series, ignore_index=True)

df_tagged


# 만든 새 데이터프레임을 엑셀로 저장
df_tagged.to_excel("insta_posed.xlsx")

### 학습 전용과 테스트 전용 분리하기 

- 학습데이터와 데스트 전용 데이터로 분할하는 메서드 
- 임포트 
    - from sklearn.model_selection import train_test_split

```
    train_test_split( data, label, 
                       test_size=float, train_size=float, 
                        shuffle=True/False)
```                    

### 데이타 학습  : 모델링

### 데이터 예측 

In [ ]:
from collections import Counter

# 테그된  리스트 담을 데이터 프레임 만들어두기
df_counts = pd.DataFrame(columns=['city', 'counts'])
df_counts.head()

for i in range(31):
    
    del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다'] 

    word_cleaned = []
    for word in df_tagged['tagged'].loc[i]:
        if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign", "Alpha","Determiner", "Number"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if (len(word[0]) != 1) & (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
                word_cleaned.append(word[0])

    word_cleaned
    word_dic = Counter(word_cleaned)
    common_words = word_dic.most_common()
    
    # 데이터프레임에 row 기준으로 업데이트
    row = [gg[i], common_words]
    row_series = pd.Series(row, index = df_counts.columns)
    df_counts = df_counts.append(row_series, ignore_index=True)
    
    
    

# 만든 새 데이터프레임을 엑셀로 저장
df_counts.to_excel("insta_counts.xlsx")
# 피클로 저장
import pickle

with open("dfcounts(insta).pickle","wb") as fw:
    pickle.dump(df_counts, fw)

### 결과 확인 

metrics.accuracy_score(label, clf.predict(data))

In [ ]:
WordCloud library  설치
> conda install -c https://conda.anaconda.org/conda-forge wordcloud==1.5.0
# WordCloud 패키지 import
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 데이터 가져오기 (df)

with open("dfcounts(insta).pickle","rb") as fr:
    data = pickle.load(fr)
    

# 100개 단어로 워드클라우드 만드는 함수 만들어보기
def make_wc(city_index:int, city:str, data: "df(city, counts_튜플담긴리스트)"):
  word_dict = {}
  for i in range(100):
    word_dict[data['counts'].loc[city_index][i][0]] = data['counts'].loc[city_index][i][1]
  mask = np.array(Image.open(city+".jpg"))
  word_cloud = WordCloud(font_path="C:/Windows/Fonts/malgun.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
                        width=2000, height=1000, max_words=150, min_font_size=5,
                        mask=mask, 
                        background_color='white').generate_from_frequencies(word_dict)

  plt.figure(figsize=(15,15))
  plt.imshow(word_cloud)
  plt.axis("off")
  plt.tight_layout(pad=0)
  plt.show()